In [1]:
import pandas as pd
import json
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('data/review_data_without_fake.csv')

In [3]:
df.business_id.value_counts().describe()

count    33780.000000
mean        22.894997
std         59.237043
min          1.000000
25%          4.000000
50%          8.000000
75%         20.000000
max       3386.000000
Name: business_id, dtype: float64

In [4]:
tmp = df[['business_id', 'pred_stars']].groupby('business_id').count().reset_index()
tmp['stars'] = df[['business_id', 'stars']].groupby('business_id').mean().values
tmp['sentiment_polarity'] = df[['business_id', 'sentiment_polarity']].groupby('business_id').mean().values
tmp['sentiment_subjectivity'] = df[['business_id', 'sentiment_subjectivity']].groupby('business_id').mean().values
tmp.columns = ['business_id', 'count', 'stars', 'sentiment_polarity', 'sentiment_subjectivity']
tmp = tmp[tmp['count'] >= 8]
tmp

,business_id,count,stars,sentiment_polarity,sentiment_subjectivity
2,--ZVrH2X2QXBFdCilbirsw,25,4.760000,0.283428,0.487427
4,--epgcb7xHGuJ-4PUeSLAw,18,3.666667,0.230327,0.520330
5,--sXnWH9Xm6_NvIjyuA99w,16,4.500000,0.300646,0.510975
7,-0FX23yAacC4bbLaGPvyxw,81,4.259259,0.344844,0.605442
10,-0TffRSXXIlBYVbb5AwfTg,620,4.648387,0.362472,0.636555
...,...,...,...,...,...
33770,zz3E7kmJI2r2JseE6LAnrw,38,3.842105,0.142438,0.458822
33771,zzFCdBSW27eKFg-xG7cqAg,9,4.333333,0.265474,0.527222
33773,zzHtFjfM7NvuVM1HTsCLGA,22,4.909091,0.323298,0.634324
33775,zzbpcMZXHoZxUr9JZdH6wg,10,1.800000,0.061583,0.412330


In [5]:
# reviews per week
df['week'] = pd.DatetimeIndex(df['date']).week
tt = df.groupby(['business_id', 'week']).count().reset_index()[['business_id', 'review_id']]
tt = tt.groupby('business_id').mean().reset_index()
tt.columns = ['business_id', 'reviews_per_week']
tmp = tmp.merge(tt, on = 'business_id', how='left')
# df.drop(columns='week', inplace=True)

In [6]:
# oldest review year
oldest_review = df[['business_id', 'year']].groupby('business_id').min().reset_index()
oldest_review.columns = ['business_id', 'oldest_review_year']
tmp = tmp.merge(oldest_review, on = 'business_id', how='left')

In [7]:
# latest review year
latest_review = df[['business_id', 'year']].groupby('business_id').max().reset_index()
latest_review.columns = ['business_id', 'latest_review_year']
tmp = tmp.merge(latest_review, on = 'business_id', how='left')

In [8]:
# std of stars
std_of_stars = df[['business_id', 'stars']].groupby('business_id').std().reset_index()
std_of_stars.columns = ['business_id', 'std_of_stars']
tmp = tmp.merge(std_of_stars, on = 'business_id', how='left')

In [9]:
# median of stars
med_of_stars = df[['business_id', 'stars']].groupby('business_id').median().reset_index()
med_of_stars.columns = ['business_id', 'med_of_stars']
tmp = tmp.merge(med_of_stars, on = 'business_id', how='left')

In [10]:
tmp

,business_id,count,stars,sentiment_polarity,sentiment_subjectivity,reviews_per_week,oldest_review_year,latest_review_year,std_of_stars,med_of_stars
0,--ZVrH2X2QXBFdCilbirsw,25,4.760000,0.283428,0.487427,1.190476,2007,2018,0.663325,5.0
1,--epgcb7xHGuJ-4PUeSLAw,18,3.666667,0.230327,0.520330,1.125000,2012,2021,1.533930,4.0
2,--sXnWH9Xm6_NvIjyuA99w,16,4.500000,0.300646,0.510975,1.142857,2011,2020,1.154701,5.0
3,-0FX23yAacC4bbLaGPvyxw,81,4.259259,0.344844,0.605442,1.840909,2018,2021,1.159502,5.0
4,-0TffRSXXIlBYVbb5AwfTg,620,4.648387,0.362472,0.636555,11.923077,2013,2022,0.826996,5.0
...,...,...,...,...,...,...,...,...,...,...
17496,zz3E7kmJI2r2JseE6LAnrw,38,3.842105,0.142438,0.458822,1.583333,2007,2022,1.197437,3.0
17497,zzFCdBSW27eKFg-xG7cqAg,9,4.333333,0.265474,0.527222,1.000000,2009,2017,1.000000,5.0
17498,zzHtFjfM7NvuVM1HTsCLGA,22,4.909091,0.323298,0.634324,1.222222,2019,2020,0.426401,5.0
17499,zzbpcMZXHoZxUr9JZdH6wg,10,1.800000,0.061583,0.412330,1.111111,2014,2021,1.398412,1.0


In [11]:
business_df = pd.read_json('data/yelp_academic_dataset_business.json', orient='record', lines=True)

In [12]:
business_df = business_df[business_df['state']=='PA']
business_df.drop(columns=['address', 'state', 'stars', 'review_count'], inplace=True)
business_df = pd.merge(business_df, tmp, on='business_id')

In [13]:
# convert dictionary in the column 'attributes' and 'hours' to labels
business_df = pd.concat([business_df, business_df["attributes"].apply(pd.Series), business_df["hours"].apply(pd.Series)], axis=1)
business_df.drop(columns=['attributes', 'hours'], inplace=True)

In [14]:
for i in ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']:
    business_df[f'{i}'] = business_df[f'{i}'].apply(lambda x: 1 if type(x) == str else 0)

In [15]:
# getting just restaurants
business_df = business_df[business_df['categories'].str.contains('Restaurant.*')==True]

In [16]:
# business_df['categories'] = business_df['categories'].str.split(', ')
# get dummies from categories
df_categories_dummies = pd.Series(business_df['categories']).str.get_dummies(',')

business_df = pd.concat([business_df, df_categories_dummies], axis=1)
business_df.drop(columns='categories', inplace=True)

In [17]:
business_df.head()

,business_id,name,city,postal_code,latitude,longitude,is_open,count,stars,sentiment_polarity,...,Vegan,Vegetarian,Venues & Event Spaces,Vietnamese,Waffles,Wedding Planning,Wheel & Rim Repair,Wine Bars,Wineries,Wraps
0,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,Philadelphia,19107,39.955505,-75.155564,1,42,4.380952,0.315001,...,0,0,0,0,0,0,0,0,0,0
2,MUTTqe8uqyMdBl186RmNeA,Tuna Bar,Philadelphia,19106,39.953949,-75.143226,1,140,4.385714,0.363286,...,0,0,0,0,0,0,0,0,0,0
3,ROeacJQwBeh05Rqg7F6TCg,BAP,Philadelphia,19147,39.943223,-75.162568,1,125,4.488000,0.306422,...,0,0,0,0,0,0,0,0,0,0
5,QdN72BWoyFypdGJhhI5r7g,Bar One,Philadelphia,19147,39.939825,-75.157447,0,37,4.567568,0.320457,...,0,0,0,0,0,0,0,0,0,0
6,Mjboz24M9NlBeiOJKLEd_Q,DeSandro on Main,Philadelphia,19127,40.022466,-75.218314,0,12,3.166667,0.113780,...,0,0,0,0,0,0,0,0,0,0


In [18]:
business_df.to_csv('data/combined.csv', index=False)